In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Preprocess

In [51]:
import collections
import jieba
import re
import json
class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None, dict_path = None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # The index for the padding word is 0
        if dict_path is not None:
            #it can only load full dict.
            f = open(dict_path,'r')
            self._token_freqs = [tuple(line.split()) for line in list(f)]
            self._token_freqs = [(item[0],int(item[-1])) for item in self._token_freqs]
        else:
            # Sort according to frequencies
            counter = count_corpus(tokens)
            self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        self.idx_to_token = ['<pad>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
        print(len(self.token_to_idx))
    def __len__(self):
        return len(self.idx_to_token)
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    @property
    def unk(self):  # Index for the unknown token
        return 0
    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):
    # Here `tokens` is a 1D list or 2D list
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # Flatten a list of token lists into a list of tokens
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

def clean_str(string):
    for ch in '!"#$&*+，、-.:；。 <=>？@[]《》^！_\|·~‘’':
        string = string.replace(ch, "")
    return string
def read_words(data_file):  #@save
    with open(data_file) as f:
        lines = f.readlines()
        lines.pop(0)
        #移除标题
    return [list(jieba.cut(clean_str(re.sub("\t\d\n","",line)))) for line in lines]

# data_file = "train_SENT.tsv"
# tokens = read_words(data_file)
# vocab = Vocab(reserved_tokens=['<unk>'],dict_path="word_freq.txt")
# vocab = Vocab(tokens,reserved_tokens=['<unk>'])
# ASAP_dict = json.dumps(vocab.token_to_idx,sort_keys=False,indent=4,separators=(',',': '))
# #保存
# fileObject = open('idx_to_token_bug.txt', 'w')
# for token in vocab.idx_to_token:
#     fileObject.write(str(token))
#     fileObject.write('\n')
# fileObject.close()
# fileObject = open('token_to_idx.json','w')
# fileObject.write(ASAP_dict)
# fileObject.close()
# fileObject = open('word_freq.txt','w')
# for freq in vocab._token_freqs:
#     fileObject.write(f"{freq[0]} {freq[1]}")
#     fileObject.write('\n')
# fileObject.close()

# Train Utils

In [52]:
import torch
import pandas as pd
import jieba
import torch.nn.functional as F
from tqdm import tqdm
from torch import nn
from torchtext.vocab import Vectors
from torch.nn.utils.rnn import pad_sequence


class CrossEntropyLoss_LS(nn.Module):
    ''' Cross Entropy Loss with label smoothing '''
    def __init__(self, label_smooth=None, class_num=5):
        super().__init__()
        self.label_smooth = label_smooth
        self.class_num = class_num
        self.eps = 1e-12
    def forward(self, pred, target):
        '''
        Args:
            pred: prediction of model output [batch_size, num_classe]
            target: ground truth of sampler [num_classes]
        '''
        if self.label_smooth is not None:
            # cross entropy loss with label smoothing
            logprobs = F.log_softmax(pred, dim=1)  # softmax + log
            target = F.one_hot(target, self.class_num)  # 转换成one-hot
            target = torch.clamp(target.float(), min=self.label_smooth / (self.class_num-1),
                                 max=1.0 - self.label_smooth)
            loss = -1 * torch.sum(target * logprobs, 1)
        else:
            # standard cross entropy loss
            loss = -1. * pred.gather(1, target.unsqueeze(-1)) + torch.log(torch.exp(pred + self.eps).sum(dim=1))
        return loss.mean()
class collector():
    def __init__(self, indexes):
        self.data = {}
        for index in indexes:
            self.data[index] = []
    def genDataFrame(self):
        return pd.DataFrame(self.data)
    def genCSV(self, name):
        pd.DataFrame(self.data).to_csv(name,encoding='utf-8',index=False)
    def append(self, item, index):
        self.data[index].append(item)
def evaluateTSV(model, state_dict_path, dataloader, loss_func, device, outputPath=None,join=False):
    model.eval()
    if outputPath is not None:
        f = open(outputPath, "w")
    if state_dict_path is not None:
        model.load_state_dict(torch.load(state_dict_path,map_location=device))
    avg_loss = 0
    correct = 0
    total = 0
    if outputPath is not None:
        f.write("index\tprediction\n")
    i = 0
    for X,keyword,y in tqdm(dataloader):
        X = X.to(device)
        y = y.to(device)
        keyword = keyword.to(device)
        output = model(X,keyword)
        test_loss = loss_func(output,y)
        avg_loss += test_loss.item()
        output = output.argmax(1)
        if join:
            output = output // 2.5
        result = output == y
        correct += result.sum().item()
        total += output.shape[0]
        if outputPath is not None:
            if output.shape == 1:
                f.write(f"{i}\t{output.item()-1}\n")
                i += 1
            else:
                for item in output:
                    f.write(f"{i}\t{item.item()-1}\n")
                    i += 1


    acc = correct/total*100
    avg_loss /= len(dataloader)
    print(f"Avg Test Loss:{avg_loss:>5.3f}")
    print(f"Accuracy: {acc:<6.2f}%")
    if outputPath is not None:
        f.close()
    model.train()
    return avg_loss,acc
def evaluateSENT(model, state_dict_path, dataloader, loss_func, device, outputPath=None, FullLabel=False, Reg=False):
    model.eval()
    if outputPath is not None:
        f = open(outputPath, "w")
    if state_dict_path is not None:
        model.load_state_dict(torch.load(state_dict_path,map_location=device))
    avg_loss = 0
    correct = 0
    total = 0
    if outputPath is not None:
        f.write("index\tprediction\tcorrect\n")
    i = 0
    for X,y in tqdm(dataloader):
        X = X.to(device)
        y = y.to(device)
        output = model(X)
        if Reg:
            if FullLabel:
                test_loss = loss_func(output.squeeze(1),y[:,0])
            else:
                test_loss = loss_func(output,y)
        else:
            output = torch.round(output)
            if FullLabel:
                test_loss = loss_func(output.squeeze(1), y[:, 0])
                result = output == y[:, 0]
            else:
                test_loss = loss_func(output, y)
                result = output == y
            correct += result.sum().item()
            total += output.shape[0]
        avg_loss += test_loss.item()
        if outputPath is not None:
            if output.shape == 1:
                f.write(f"{i}\t{output.item()}\n")
                # f.write(f"{i}\t{output.item()}\t{y.squeeze(1).item()}\n")
                i += 1
            else:
                for item in output:
                    f.write(f"{i}\t{item.item()}\n")
                    # f.write(f"{i}\t{output.item()}\t{y.squeeze(1).item()}\n")
                    i += 1
    avg_loss /= len(dataloader)
    print(f"Avg Test Loss:{avg_loss:>5.3f}")
    if not Reg:
        acc = correct / total * 100
        print(f"Accuracy: {acc:<6.2f}%")
    if outputPath is not None:
        f.close()
    model.train()
    if Reg:
        return avg_loss
    else:
        return avg_loss,acc
def evaluateSENT_CLS(model, state_dict_path, dataloader, loss_func, device, outputPath=None, FullLabel=False):
    model.eval()
    if outputPath is not None:
        f = open(outputPath, "w")
    if state_dict_path is not None:
        model.load_state_dict(torch.load(state_dict_path,map_location=device))
    avg_loss = 0
    correct = 0
    total = 0
    if outputPath is not None:
        f.write("index\tprediction\tlabel\n")
    i = 0
    for X,y in tqdm(dataloader):
        X = X.to(device)
        y = y = (y-1).to(device).to(torch.long)
        output = model(X)
        predict = output.argmax(1)
        if FullLabel:
            test_loss = loss_func(output, y[:, 0])
            result = predict == y[:, 0]
        else:
            test_loss = loss_func(output, y)
            result = predict == y
        correct += result.sum().item()
        total += output.shape[0]
        avg_loss += test_loss.item()
        if outputPath is not None:
            if predict.shape == 1:
                f.write(f"{i}\t{predict.item()}\t{y[0][0].item()}\n")
                i += 1
            else:
                for item in predict:
                    f.write(f"{i}\t{item.item()}\t{y[0][0].item()}\n")
                    i += 1
    avg_loss /= len(dataloader)
    print(f"Avg Test Loss:{avg_loss:>5.3f}")
    acc = correct / total * 100
    print(f"Accuracy: {acc:<6.2f}%")
    if outputPath is not None:
        f.close()
    model.train()
    return avg_loss,acc
class ASAP_ASPECT_DICT(torch.utils.data.Dataset):
    def __init__(self, data_file, dict_obj, aspect_dict,mode="train"):
        self.raw_data = pd.read_csv(data_file,sep='\t')
        self.vocab = dict_obj
        self.aspect_dict = aspect_dict
        self.label_dict = {"text_a":1,"cate":2,"label":3}
        self.mode = mode

    def __len__(self):
        return len(self.raw_data)
    def padVector(self):
        return 0
    def clean_str(self, string):
        for ch in '!"#$&*+，、-.:；<=>？@[]《》^_|·~‘’':
            string = string.replace(ch,"")
        return string
    def __getitem__(self, idx):
        line = self.raw_data.iloc[idx]
        indexes = [self.label_dict["cate"],self.label_dict["label"],self.label_dict["text_a"]]
        if self.mode == "train":
            for i in range(len(indexes)):
                indexes[i] -= 1
        keyword = self.aspect_dict[line[indexes[0]]]
        label = torch.tensor(line[indexes[1]])
        seg_list = list(jieba.cut(self.clean_str(line[indexes[2]])))
        sentence = []
        for word in seg_list:
            if word in self.vocab.token_to_idx:
                vector_index = self.vocab[word]
                sentence.append(vector_index)
            else:
                # For Unknown words.
                sentence.append(1)
        return torch.as_tensor(sentence), label+1, keyword
class ASAP_WIKI(torch.utils.data.Dataset):
    def __init__(self, data_file, vec_file, max_seq_length,aspect_dict,mode="train"):
        self.raw_data = pd.read_csv(data_file,sep='\t')
        self.wordvec = Vectors(name=vec_file)
        self.max_seq_length = max_seq_length
        self.aspect_dict = aspect_dict
        self.label_dict = {"text_a":1,"cate":2,"label":3}
        self.mode = mode

    def __len__(self):
        return len(self.raw_data)
    def padVector(self):
        return self.wordvec.stoi['。']
    def clean_str(self, string):
        for ch in '!"#$&*+，、-.:；<=>？@[]《》^_|·~‘’':
            string = string.replace(ch,"")
        return string
    def __getitem__(self, idx):
        line = self.raw_data.iloc[idx]
        indexes = [self.label_dict["cate"],self.label_dict["label"],self.label_dict["text_a"]]
        if self.mode == "train":
            for i in range(len(indexes)):
                indexes[i] -= 1
        keyword = self.aspect_dict[line[indexes[0]]]
        label = torch.tensor(line[indexes[1]])
        seg_list = list(jieba.cut(self.clean_str(line[indexes[2]])))
        sentence = []
        for word in seg_list[:min(len(seg_list),self.max_seq_length)]:
            if word in self.wordvec.stoi:
                vector_index = self.wordvec.stoi[word]
                sentence.append(vector_index)
            else:
                sentence.append(0)
        return torch.tensor(sentence), label+1, keyword
class ASAP_Full(torch.utils.data.Dataset):
    def __init__(self, data_file, dictObj):
        self.raw_data = pd.read_csv(data_file)
        self.lookupTable = dictObj;

    def __len__(self):
        return len(self.raw_data)
    def clean_str(self, string):
        for ch in '!"#$&*+，、-.:；<=>？@[]《》^_|·~‘’':
            string = string.replace(ch,"")
        return string
    def __getitem__(self, idx):
        line = self.raw_data.iloc[idx]
        label = torch.tensor(line[2:])
        seg_list = list(jieba.cut(self.clean_str(line[1])))
        sentence = []
        for word in seg_list:
            if word in self.lookupTable.token_to_idx:
                vector_index = self.lookupTable[word]
                sentence.append(vector_index)
            else:
                sentence.append(1)
        mask = label > -2
        return torch.tensor(sentence), label * mask
class ASAP_SENT_DICT(torch.utils.data.Dataset):
    def __init__(self, data_file, dict_obj, mode="train"):
        self.raw_data = pd.read_csv(data_file, sep='\t')
        self.label_dict = {"text_a": 1, "star": 2}
        self.vocab = dict_obj
        self.mode = mode

    def __len__(self):
        return len(self.raw_data)

    def padVector(self):
        return 0

    def clean_str(self, string):
        for ch in '!"#$&*+，、-.:；<=>？@[]《》^_|·~‘’':
            string = string.replace(ch, "")
        return string

    def __getitem__(self, idx):
        line = self.raw_data.iloc[idx]
        indexes = [self.label_dict["text_a"], self.label_dict["star"]]
        if self.mode == "train":
            for i in range(len(indexes)):
                indexes[i] -= 1
        if self.mode == "test":
            label = torch.tensor(0)
        else:
            label = torch.tensor(line[indexes[1]])
        seg_list = list(jieba.cut(self.clean_str(line[indexes[0]])))
        sentence = []
        for word in seg_list:
            if word in self.vocab.token_to_idx:
                vector_index = self.vocab[word]
                sentence.append(vector_index)
            else:
                #For Unknown words.
                sentence.append(1)
        # if len(seg_list)<self.max_seq_length:
        #     sentence = sentence + [self.padVector()]*(self.max_seq_length-len(seg_list))
        return torch.as_tensor(sentence), label
def collcate_pad(data):
    reviews = [item[0] for item in data]
    reviews = torch.as_tensor(pad_sequence(reviews,batch_first=True))
    labels = torch.as_tensor([item[1] for item in data])
    cates = torch.as_tensor([item[2] for item in data])
    return reviews,cates,labels
def collcate_pad_SENT(data):
    reviews = [item[0] for item in data]
    reviews = torch.as_tensor(pad_sequence(reviews, batch_first=True))
    labels = torch.tensor([item[1] for item in data])
    return reviews, labels
def collcate_pad_Full(data):
    reviews = [item[0] for item in data]
    reviews = torch.as_tensor(pad_sequence(reviews, batch_first=True))
    labels = torch.cat([item[1] for item in data]).reshape(len(data),-1)
    return reviews,labels
def get_aspect_vector_DICT(vocabObj):
    aspect_map = {'Location#Transportation':"交通", 'Location#Downtown':"商圈",
                   'Location#Easy_to_find':"定位", 'Service#Queue':"排队", 'Service#Hospitality':"服务",
                   'Service#Parking':"停车", 'Service#Timely':"准时", 'Price#Level':"价格",
                   'Price#Cost_effective':"性价比", 'Price#Discount':"折扣", 'Ambience#Decoration':"装修",
                   'Ambience#Noise':"噪音", 'Ambience#Space':"空间", 'Ambience#Sanitary':"卫生", 'Food#Portion':"分量",
                   'Food#Taste':"口味", 'Food#Appearance':"外观", 'Food#Recommend':"推荐"}
    aspect_indexes = []
    wordvec_dict = {}
    for key in aspect_map:
        indexKey = torch.tensor(vocabObj.token_to_idx[aspect_map[key]])
        aspect_indexes.append(indexKey)
        wordvec_dict[key] = indexKey
    return aspect_indexes,wordvec_dict

# Model

In [53]:
from torchtext.vocab import Vectors
import torch
import torch.nn.functional as F
from torch import nn
class GCAE_ACSA(nn.Module):
    def __init__(self, vocab_size, embed_size, kernel_sizes, num_channels, aspect_indexes, num_classes=3):
        super(GCAE_ACSA, self).__init__()
        self.embedding1 = nn.Embedding(vocab_size, embed_size)
        self.embedding2 = nn.Embedding(vocab_size, embed_size)
        self.aspectFC = nn.Linear(2 * embed_size,num_channels[0])
        self.sentConv = nn.ModuleList()
        self.aspectConv = nn.ModuleList()
        for c, k in zip(num_channels, kernel_sizes):
            self.sentConv.append(nn.Conv1d(2 * embed_size, c, k))
            self.aspectConv.append(nn.Conv1d(2 * embed_size, c, k))
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.classifyFC = nn.Linear(sum(num_channels),num_classes)
        self.aspectIndexes = aspect_indexes
    def forward(self, x, key=None):
        embeddings = torch.cat((self.embedding1(x), self.embedding2(x)), dim=2).transpose(1,2)
        sent_feat = torch.cat([torch.squeeze(torch.tanh(self.pool(conv(embeddings))), dim=-1)
            for conv in self.sentConv], dim=1)
        if key is not None:
            aspect_embed = torch.cat((self.embedding1(key), self.embedding2(key)), dim=1)
            aspect_feat = torch.cat([self.relu(torch.squeeze(self.pool(conv(embeddings)), dim=-1)+self.aspectFC(aspect_embed)) for conv in self.aspectConv], dim=1)
            out = sent_feat * aspect_feat
        else:
            out = sent_feat
        out = self.classifyFC(self.dropout(out))
        return out
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, kernel_sizes, num_channels,
                 **kwargs):
        super(TextCNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # The embedding layer not to be trained
        self.constant_embedding = nn.Embedding(vocab_size, embed_size)
        self.dropout = nn.Dropout(0.5)
        self.decoder = nn.Linear(sum(num_channels), 3)
        # The max-over-time pooling layer has no parameters, so this instance
        # can be shared
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.relu = nn.ReLU()
        # Create multiple one-dimensional convolutional layers
        self.convs = nn.ModuleList()
        for c, k in zip(num_channels, kernel_sizes):
            self.convs.append(nn.Conv1d(2 * embed_size, c, k))

    def forward(self, inputs, dummy=None):
        # Concatenate two embedding layer outputs with shape (batch size, no.
        # of tokens, token vector dimension) along vectors
        embeddings = torch.cat((
            self.embedding(inputs), self.constant_embedding(inputs)), dim=2)
        # Per the input format of one-dimensional convolutional layers,
        # rearrange the tensor so that the second dimension stores channels
        embeddings = embeddings.permute(0, 2, 1)
        # For each one-dimensional convolutional layer, after max-over-time
        # pooling, a tensor of shape (batch size, no. of channels, 1) is
        # obtained. Remove the last dimension and concatenate along channels
        encoding = torch.cat([
            torch.squeeze(self.relu(self.pool(conv(embeddings))), dim=-1)
            for conv in self.convs], dim=1)
        outputs = self.decoder(self.dropout(encoding))
        return outputs
def init_weights(item):
    if isinstance(item,nn.Conv1d) or isinstance(item,nn.Linear):
        torch.nn.init.xavier_uniform_(item.weight)
        item.bias.data.fill_(0.01)
    if type(item) == nn.LSTM:
        for param in item._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(item._parameters[param])

# Train Function

In [54]:
def train_ASPECT(model, dataloader, test_dataloader, loss_func, optimizer, epochs, device, save_name, logger,join = False):
    t0 = time.time()
    plot_loss = collector(["epoch","train_loss","val_loss"])
    plot_acc = collector(["epoch","acc"])
    model = model.to(device)
    best_acc = -1
    best_epoch = -1
    for epoch in range(epochs):
        print('='*10 + f"epoch {epoch+1}" + '='*10)
        avg_loss = 0
        step = 0
        total = len(dataloader)
        for X,keyword,y in tqdm(dataloader):
            X = X.to(device)
            y = y.to(torch.long).to(device)
            keyword = keyword.to(device)
            output = model(X,keyword)
            if join:
                for i in range(y.shape[0]):
                    if y[i]==1:
                        y[i] = torch.randint(1,3,(1,))
                    elif y[i]==2:
                        y[i] = torch.randint(3,5,(1,))
            train_loss = loss_func(output,y)
            avg_loss += train_loss.item()
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            step += 1
            if step%50==0:
                print(f"Step {step}/{total} Batch Loss: {train_loss.item():>5.3f}")
                logger.info(f"Step {step}/{total} Batch Loss: {train_loss.item():>5.3f}")
        avg_loss /= len(dataloader)
        print(f"Avg Train Loss of Epoch {epoch+1}: {avg_loss:>5.3f}")
        logger.info(f"Avg Train Loss of Epoch {epoch+1}:{avg_loss:>5.3f}")
        result = evaluateTSV(model, None ,test_dataloader, loss_func, device, join=False)
        plot_loss.append(epoch+1,"epoch")
        plot_loss.append(avg_loss,"train_loss")
        plot_loss.append(result[0],"val_loss")
        plot_acc.append(epoch+1,"epoch")
        plot_acc.append(result[1],"acc")
        logger.info(f"Acc: {result[1]:<6.2f}%")
        if result[1]>best_acc:
            best_acc = result[1]
            best_epoch = epoch + 1
            bestCallOut = f"Best acc {best_acc:<6.2f}% is at Epoch {best_epoch}"
            print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + bestCallOut)
            logger.info(bestCallOut)
            torch.save(model.state_dict(), save_name + '.pt')
            plot_acc.genCSV(save_name + " acc_evo.csv")
        plot_loss.genCSV(save_name + " loss_evo.csv")
    print("Training Completed. With " + bestCallOut + f" Total Time:{time.time() - t0:>5.2f}s")


# Main

In [ ]:
import torch
import logging
import time
import datetime
from torch import nn
from torch.utils.data import DataLoader
#Config Logger
datasetName = "trian.tsv"
save_name = datasetName + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
Log_Format = "%(levelname)s %(asctime)s - %(message)s"
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(
    filename=save_name + '.log',
    filemode="w",
    format=Log_Format,
    level=logging.INFO)
logger = logging.getLogger()

#Environment Configuration
logger.info(torch.__version__)
import numpy as np
logger.info(np.__version__)
import sys
logger.info(sys.version)

#Hyperparameters
wordvec_length = 50
epochs = 30
random_seed = "random"
# random_seed = 500
# torch.manual_seed(random_seed)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# wordvec_path = "/Users/Charlie/Desktop/sgns.wiki.bigram"
gpu_boost = 1
base_lr = 4e-4
learning_rate = base_lr*gpu_boost if device != 'cpu' else base_lr
base_batch = 64
batch_size = base_batch*gpu_boost if device != 'cpu' else base_batch
pool_length = 1
num_aspects = 18
min_freq = 5
# wiki_vec_dict = 352217
input_path = "/kaggle/input/meituan-asap/"
loss_func = CrossEntropyLoss_LS(label_smooth=0.10,class_num=3)
kernel_sizes, num_channels = [3, 4, 5], [64, 64, 64]
#Load Data
vocab = Vocab(reserved_tokens=['<unk>'],min_freq=min_freq,dict_path="/kaggle/input/meituan-asap-sent/word_freq.txt")
aspect_indexes,aspect_dict = get_aspect_vector_DICT(vocab)
aspect_indexes = torch.tensor(aspect_indexes).to(device)
training_data = ASAP_ASPECT_DICT(input_path + datasetName,vocab,aspect_dict)
test_data = ASAP_ASPECT_DICT(input_path + "dev.tsv",vocab,aspect_dict,"val")
train_dataloader = DataLoader(training_data, batch_size=batch_size, \
collate_fn=collcate_pad, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=1, collate_fn=collcate_pad)
#Prepare Model
NN = TextCNN(len(vocab),wordvec_length,kernel_sizes,num_channels)
NN.apply(init_weights)
optimizer = torch.optim.Adam(NN.parameters(),lr=learning_rate)
#Start Training
print('=' * 7 + "Training Begin" + '=' * 7)
print(
    f"total epoch: {epochs}\ndevice: {device}\nbatch size: {batch_size}\nloss func:{loss_func}\nlr: {learning_rate}\nrandom_seed: {random_seed}\ndict size: {len(vocab)}\nwordvec length: {wordvec_length}\nfilter num: {num_channels}\nfilter size: {kernel_sizes}\npool length: {pool_length}")
logger.info(
    f"total epoch: {epochs}\ndevice: {device}\nbatch size: {batch_size}\nloss func:{loss_func}\nlr: {learning_rate}\nrandom_seed: {random_seed}\ndict size: {len(vocab)}\nwordvec length: {wordvec_length}\nfilter num: {num_channels}\nfilter size: {kernel_sizes}\npool length: {pool_length}")
logger.info(NN)
train_ASPECT(NN,train_dataloader,test_dataloader,loss_func,optimizer,epochs,device,save_name,logger,join=False)